# Import library

In [25]:
# get data
from binance.spot import Spot 

# train data
import pandas as pd
from datetime import datetime

# enviroment
import os
from dotenv import load_dotenv
load_dotenv("../../env/app.env")

True

# Setup clients

In [26]:
BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_API_SECRET = os.environ.get("BINANCE_API_SECRET")
client = Spot(key=BINANCE_API_KEY, secret=BINANCE_API_SECRET)

# Fetch day period data

### Declare period, symbol and header

In [27]:
DAY_TO_QUERY = 1800
DURATION_EACH_DAY = 60 * 60 * 24 * 1000

In [28]:
PERIOD = "1d"
cols =[
        'Kline open time',   #if a range of time
        'Open price',       #First trade price in the time range
        'High price',       #max buy in time
        'Low price',        #Min sell in time
        'Close price',      #Last trade price in the time range
        'Volume',           #Sum in time
        'Kline Close time',
        'Quote asset volume',   #Sum in time
        'Number of trades',     #Sum in time
        'Taker buy base asset volume',  #Sum in time
        'Taker buy quote asset volume', #Sum in time
        'Unused field, ignore'] # not use 

### Get data points of symbols 

In [29]:
exchanges = client.exchange_info()
symbols = list(map(lambda item: item.get('symbol'), exchanges.get('symbols')))

In [30]:
print(len(symbols))
symbols[:10]

2137


['ETHBTC',
 'LTCBTC',
 'BNBBTC',
 'NEOBTC',
 'QTUMETH',
 'EOSETH',
 'SNTETH',
 'BNTETH',
 'BCCBTC',
 'GASBTC']

In [31]:
print(f"{len(symbols)} symbol X {DAY_TO_QUERY} data points = {DAY_TO_QUERY*len(symbols)} data points")

2137 symbol X 1800 data points = 3846600 data points


In [32]:
today = datetime.utcnow().replace(hour=23, minute=0, second=0, microsecond=0)
today_timestamp = int(today.timestamp() * 1000)

start_timestamp = today_timestamp - DAY_TO_QUERY * DURATION_EACH_DAY
end_timestamp = today_timestamp

for symbol in ["BTCUSDT"]:
    try:
        for timestamp in range(start_timestamp, end_timestamp, DURATION_EACH_DAY * 100):
            data = client.klines(symbol, PERIOD, limit=100, startTime=timestamp, endTime=timestamp + DURATION_EACH_DAY * 100)
            if start_timestamp == timestamp:
                df = pd.DataFrame(data, columns=cols)
            else:
                df = pd.concat([df, pd.DataFrame(data, columns=cols)], axis=0)

        df.drop("Unused field, ignore", axis=1, inplace=True)
        df.to_csv(f"../../datastore/price/{symbol}_{PERIOD}.csv", index=False)
    except:
        print(f"Error at {symbol}")